In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets.karate import KarateClub

FileNotFoundError: Could not find module 'C:\Users\PC\anaconda3\envs\glw-examples\Lib\site-packages\torch_sparse\_convert_cuda.pyd' (or one of its dependencies). Try using the full path with constructor syntax.